<a href="https://colab.research.google.com/github/huangjihui511/BUAASE2020_IntersectProject/blob/master/main_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
path = "COVID-Dialogue-Dataset-English.txt"
f = open(path,encoding="UTF-8")
l1 = f.read()
l2 = l1.split("id=")
p2d = []
d2p = []
for item in l2:
    if len(item) != 0:
        temp = item.split("Dialogue\nPatient:")
        if len(temp) > 1:
            item2 = temp[1]
            item3 = item2.split("Patient:")
            l = []
            if len(item3) != 0:
                for item4 in item3:
                    item5 = item4.split("Doctor:")
                    l += item5
            for i in range(1, len(l), 2):
                p2d.append((l[i - 1].strip("\n"), l[i].strip("\n")))
                if i + 1 < len(l):
                    d2p.append((l[i].strip("\n"), l[i + 1].strip("\n")))
data = p2d + d2p
data[0]

('Hello doctor, I get a cough for the last few days, which is heavy during night times. No raise in temperature but feeling tired with no travel history. No contact with any Covid-19 persons. It has been four to five days and has drunk a lot of Benadryl and took Paracetamol too. Doctors have shut the OP so do not know what to do? Please help.',
 'Hello, I understand your concern. I just have a few more questions.\nDoes your cough has phlegm? Any other symptoms like difficulty breathing? Any other medical condition such as asthma, hypertension? Are you a smoker? Alcoholic beverage drinker?')

In [0]:
dictionary = {}

import nltk
nltk.download('stopwords')

from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from itertools import chain

sw = stopwords.words('english')
nltk.download('punkt')
punct = punctuation
reverse_flag = True

def clean_data(data):
    for item in data:
        for item2 in item:
            words = word_tokenize(item2)
            for word in words:
                dictionary.setdefault(word,0)
                dictionary[word] += 1
    data2 = []
    for item in data:
        temp = []
        for item2 in item:
            item2 = " ".join([token for token in word_tokenize(item2) if dictionary[token] > 0 and token.lower() not in chain(punct, sw)])
            if reverse_flag:
                item2 = " ".join(reversed(item2.split(" ")))
            temp.append(item2)
        temp = tuple(temp)
        data2.append(temp)
    return data2
data2 = clean_data(data)
data2[0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


('help Please know OP shut Doctors Paracetamol took Benadryl lot drunk days five four persons Covid-19 contact history travel tired feeling temperature raise times night heavy days last cough get doctor Hello',
 'drinker beverage Alcoholic smoker hypertension asthma condition medical breathing difficulty like symptoms phlegm cough questions concern understand Hello')

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [0]:
def readLangs(lang):
    print("Reading lines...")
    '''
    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    '''
    pairs = [(normalizeString(item[0]), normalizeString(item[1])) for item in data2]
    # Reverse pairs, make Lang instances


    lang = Lang(lang)

    return lang, pairs

In [0]:
MAX_LENGTH = 1000

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [0]:
def prepareData(lang):
    lang, pairs = readLangs(lang)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        #print(pair)
        lang.addSentence(pair[0])
        lang.addSentence(pair[1])
    print("Counted words:")
    print(lang.name, lang.n_words)
    return lang, pairs


lang, pairs = prepareData('lang')
print(random.choice(pairs))

Reading lines...
Read 628 sentence pairs
Trimmed to 628 sentence pairs
Counting words...
Counted words:
lang 5283
('covid tested medication responding days headaches throat sore degrees around temperature high daughter', 'w video maybe')


In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(lang, pair[0])
    target_tensor = tensorFromSentence(lang, pair[1])
    return (input_tensor, target_tensor)

In [0]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [0]:
hidden_size = 128
encoder1 = EncoderRNN(lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 10000, print_every=500)

0m 9s (- 31m 53s) (50 0%) 5.4034
0m 16s (- 27m 1s) (100 1%) 4.0897
0m 25s (- 27m 57s) (150 1%) 4.7177
0m 33s (- 27m 24s) (200 2%) 4.5598
0m 43s (- 28m 32s) (250 2%) 5.3923
0m 55s (- 29m 48s) (300 3%) 5.9856
1m 3s (- 29m 13s) (350 3%) 4.4751
1m 16s (- 30m 35s) (400 4%) 5.4752
1m 26s (- 30m 37s) (450 4%) 5.2837
1m 34s (- 29m 58s) (500 5%) 4.9294
1m 43s (- 29m 37s) (550 5%) 4.2398
1m 53s (- 29m 31s) (600 6%) 5.0610
2m 5s (- 29m 59s) (650 6%) 5.7830
2m 14s (- 29m 44s) (700 7%) 4.9981
2m 23s (- 29m 34s) (750 7%) 4.8577
2m 32s (- 29m 16s) (800 8%) 5.2678
2m 42s (- 29m 8s) (850 8%) 5.0635
2m 50s (- 28m 46s) (900 9%) 4.7494
3m 1s (- 28m 51s) (950 9%) 5.4268
3m 12s (- 28m 49s) (1000 10%) 5.1545
3m 23s (- 28m 53s) (1050 10%) 5.1437
3m 33s (- 28m 50s) (1100 11%) 5.3195
3m 45s (- 28m 54s) (1150 11%) 5.1719
3m 53s (- 28m 29s) (1200 12%) 4.5779
4m 3s (- 28m 26s) (1250 12%) 4.8539
4m 12s (- 28m 11s) (1300 13%) 4.8452
4m 22s (- 28m 4s) (1350 13%) 4.9664
4m 34s (- 28m 7s) (1400 14%) 5.1022
4m 45s (- 28

KeyboardInterrupt: ignored

In [0]:
evaluateRandomly(encoder1, attn_decoder1)

> infection avoid take precautions water spreads covid morning daily swim use old years chaitanya hi
= vaishalee dr assist know let helps hope travel avoid sickness home stay mask face cover water soap frequently hands wash gatherings social avoid include precautions minutes host find doesnot pool swimming killed get virus although sometime swimming refrain suggest would droplets air mainly spread human human however food spread suspected cases first advice query reviewed service doctor ask welcome hello
< chat text video like would covid doctor doctor doctor covid <EOS>

> chest sensation burning really sure university gibs covid exposure possible fever worse s upright stand cough sometimes cough like feels cough chest tight hi
= chat text video like would medication ekg x ray chest possible need assess lungs listen person perhaps visit video assp seen worsens breathing recommendation office dr call best probably likely less covid cough fever without problems heart certain breath shor

In [0]:
lang.index2word

{0: 'SOS',
 1: 'EOS',
 2: 'hello',
 3: 'doctor',
 4: 'get',
 5: 'cough',
 6: 'last',
 7: 'days',
 8: 'heavy',
 9: 'night',
 10: 'times',
 11: 'raise',
 12: 'temperature',
 13: 'feeling',
 14: 'tired',
 15: 'travel',
 16: 'history',
 17: 'contact',
 18: 'covid',
 19: 'persons',
 20: 'four',
 21: 'five',
 22: 'drunk',
 23: 'lot',
 24: 'benadryl',
 25: 'took',
 26: 'paracetamol',
 27: 'doctors',
 28: 'shut',
 29: 'op',
 30: 'know',
 31: 'please',
 32: 'help',
 33: 'understand',
 34: 'concern',
 35: 'questions',
 36: 'phlegm',
 37: 'symptoms',
 38: 'like',
 39: 'difficulty',
 40: 'breathing',
 41: 'medical',
 42: 'condition',
 43: 'asthma',
 44: 'hypertension',
 45: 'smoker',
 46: 'alcoholic',
 47: 'beverage',
 48: 'drinker',
 49: 'thank',
 50: 'tiny',
 51: 'amount',
 52: 'comes',
 53: 'time',
 54: 'conditions',
 55: 'hi',
 56: 'would',
 57: 'recommend',
 58: 'take',
 59: 'n',
 60: 'acetylcysteine',
 61: 'mg',
 62: 'powder',
 63: 'dissolved',
 64: 'water',
 65: 'three',
 66: 'day',
 67: 'm

In [0]:
lang.n_words

5283